In [6]:
import cv2
import os
import numpy as np

def detect_aruco_markers_and_save(image_path, save_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(image, (5, 5), 0)
    
    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)
    parameters = cv2.aruco.DetectorParameters_create()
    corners, ids, rejected_img_points = cv2.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    image_with_markers = cv2.aruco.drawDetectedMarkers(image.copy(), corners, ids, (0, 255, 0))
    height, width, _ = image.shape
    save_filename = os.path.join(save_path, 'detected_markers_image.jpg')
    cv2.imwrite(save_filename, image_with_markers)
    print(f"Image with detected markers saved at: {save_filename}")

    
    center_points = []
    for (marker, id) in zip(corners, ids):
        corner = marker.reshape((4, 2))
        (tl, tr, br, bl) = corner
        cx = int((tl[0] + br[0]) / 2)
        cy = int((tl[1] + br[1]) / 2)
        center_points.append((cx, cy))
        cv2.circle(image, (cx, cy), 4, (255, 1, 1), 2)  
        label_text = f'ID:{id[0]} ({cx},{cy})'
        cv2.putText(image, label_text, (cx + 25, cy - 25), cv2.FONT_HERSHEY_SIMPLEX, 2, (1, 1, 1), 1, cv2.LINE_AA)

    center_points = np.array(center_points)
    hull = cv2.convexHull(center_points, clockwise=True, returnPoints=False)
    rect = cv2.minAreaRect(center_points)
    box = cv2.boxPoints(rect)
    box = np.int0(box)
    area = cv2.contourArea(box)
    

    (center, size, angle) = rect
    (width, height) = size
    print(center)
    
    print(f"Length of the rectangle: {width} pixels")
    print(f"Breadth of the rectangle: {height} pixels")

    mask = np.zeros_like(image)
    cv2.drawContours(mask, [box], 0, (255, 255, 255), 1)
    result_cropped = cv2.bitwise_and(image, mask)

    center = (int(rect[0][0]), int(rect[0][1]))
    radius = 50
    cv2.circle(image, center, radius, (123, 222, 1), 5)
    print(area)
    
    box_image = np.zeros_like(image)
    cv2.drawContours(box_image, [box], 0, (122,250, 243), -1)

    opacity = 0.10
    cv2.addWeighted(box_image, opacity, image, 1 - opacity, 0, image)
    
    
    # Save the cropped image
    save_filename_cropped = os.path.join(save_path, 'cropped_image.jpg')
    cv2.imwrite(save_filename_cropped, result_cropped)
    print(f"Cropped image inside the resultant box saved at: {save_filename_cropped}")
    save_filename_with_circles_and_box = os.path.join(save_path, 'image_with_circles_box.jpg')
    cv2.imwrite(save_filename_with_circles_and_box, image)
    print(f"Image with circles, blue box, and lines saved at: {save_filename_with_circles_and_box}")

image_path = '/home/qt/visionSort/visionSort_cv/Alignment Page.jpg'
save_path = '/home/qt/visionSort/visionSort_cv/imgs'
detect_aruco_markers_and_save(image_path, save_path)

Image with detected markers saved at: /home/qt/visionSort/visionSort_cv/imgs/detected_markers_image.jpg
(1647.5, 1275.5)
Length of the rectangle: 1945.0 pixels
Breadth of the rectangle: 2695.0 pixels
5241775.0
Cropped image inside the resultant box saved at: /home/qt/visionSort/visionSort_cv/imgs/cropped_image.jpg
Image with circles, blue box, and lines saved at: /home/qt/visionSort/visionSort_cv/imgs/image_with_circles_box.jpg


In [7]:
import numpy as np

def set_robot_position(x, y, z, t):
    center_x = 1650
    center_y = 1272
    translation_z = -163.0
    
    translation_matrix = np.array([
        [1, 0, 0, center_x],
        [0, 1, 0, center_y - -163],
        [0, 0, 1, translation_z],
        [0, 0, 0, 1]
    ])

    position_vector = np.array([x, y, z, 1])

    translated_position = np.dot(translation_matrix, position_vector)

    translated_x, translated_y, translated_z, _ = translated_position

    for i in range(len(translated_position)):
        print(translated_position[i])

# Example usage:
set_robot_position(-1350, -1132, 0, 1)


300.0
303.0
-163.0
1.0


In [8]:
l = 51.461458333
b = 71.305208333

In [9]:
l / 2

25.7307291665

In [10]:
b / 2

35.6526041665

In [11]:
356.525 - 162

194.52499999999998

In [12]:
1945/514.61

3.7795612211189056

In [13]:
1647.5 / 3.77


437.0026525198939

In [14]:
1275.5 / 3.77

338.3289124668435

In [15]:
437 - 300, 338.32 - 303

(137, 35.31999999999999)

In [20]:
972.5 / 8

121.5625

In [21]:
1347.5 / 11

122.5

In [17]:
1647.5 - 300, 1275.5 - 303

(1347.5, 972.5)

In [23]:
-164 - 122

-286

In [27]:
(1647.5 - 300)/122 * 10, (1275.5 - 2248)/122 * 10

(110.45081967213115, -79.71311475409836)

In [47]:
def calculate_scaling_factor(x, y, length_proportion, breadth_proportion, image_center=(1647.5, 1275.5)):
    factor_x = (image_center[0] - x) / length_proportion
    factor_y = (image_center[1] - y) / breadth_proportion
    scaling_factor = (factor_x + factor_y) / 2
    return scaling_factor
    
def calculate_relative_coordinates(x, y, scaling_factor, image_center=(1647.5, 1275.5)):
    relative_x = (image_center[0] - x) / scaling_factor
    relative_y = (image_center[1] - y) / scaling_factor
    return relative_x, relative_y


In [48]:
s_f = calculate_scaling_factor(300,303,110,80)

In [51]:
r_c = calculate_relative_coordinates(2626,1118, s_f)

In [52]:
r_c

(-80.1843790012804, 12.906530089628681)